In [9]:
%pip install numpy
%pip install scipy
%pip install plotly

import numpy as np
from scipy.optimize import minimize_scalar

# Constants in SI units
Trs = 298.15  # Reference temperature in K
Prs = 100000 # Reference pressure in Pa
S_IG = -40.87294625959575
a1 = 9.48*4.184  # J/mol
b1 = 0.00914*4.184  # J/(mol*K)
c1 = -6.4e-7*4.184  # J/(mol*K^2)

R = 8.314  # J/(mol*K)
Tc = 414.48  # K
Pc = 6.8e6  # Pa
w = 0.215  # dimensionless

# Peng-Robinson equation parameters
a = 0.45724 * (R**2 * Tc**2) / Pc  # Pa * m^6 / mol^2
b = 0.07780 * R * Tc / Pc  # m^3 / mol


# Define the pressure function
def calculate_pressure(Vm, T):
    k = 0.37464 + 1.54226*w - 0.26992*w**2 
    alpha = (1 + k * (1 - np.sqrt(T / Tc)))**2
    return (R * T / (Vm - b)) - (a * alpha / (Vm**2 + 2 * b * Vm - b**2))
# Pre-calculated on input parameters
def calc_a(T):
    kappa = 0.37464 + 1.54226*w - 0.26992*w**2     # Peng-Robinson kappa
    return 0.45724 * R**2 * Tc**2 / Pc * (1 + kappa * (1 - np.sqrt(T/Tc)))**2

def calc_b():
    return 0.07780 * R * Tc / Pc

# Compressibility factor cubic equation coefficients
def cubic_eq_coeffs(A, B):
    return [1, -(1 - B), A - 3*B**2 - 2*B, -(A*B - B**2 - B**3)]

# Solving for Z (compressibility factor)
def solve_cubic(A, B):
    coeffs = cubic_eq_coeffs(A, B)
    roots = np.roots(coeffs)
    roots_real = np.real(roots[np.isreal(roots)])   # Only real roots
    return roots_real 

# Calculating Z, V, H, S
def calculate_properties(T, P):
    A = calc_a(T) * P / (R * T)**2
    B = calc_b() * P / (R * T)
    Z = solve_cubic(A, B)

    Z1 = min(Z)
    Z3 = max(Z)

    # Molar volume V = ZRT/P
    V1 = 1000 * Z1 * R * T / P
    V3 = 1000 * Z3 * R * T / P
    return Z1 , Z3, V1, V3

# Integrals for Cp and Cp/T
def integral_Cp_by_T(T, P):
    term1 = a1 *np.log(T / Trs)
    term2 = (b1) * (T - Trs)
    term3 = (c1 / 2) * (T**2 - Trs**2)
    term4 = -1*R*np.log(P / 101325)
    term5 = S_IG
    return term1 + term2 + term3 + term4 + term5

def d_alpha(T):
    k = 0.37464 + 1.54226*w - 0.26992*w**2    # Peng-Robinson k
    return - (1 + k*(1 - np.sqrt(T/Tc))) * ( k / Tc * np.sqrt(Tc/T))

def delta_S(T, P):
       
    b = calc_b()
    B = b * P / (R * T)
    Z1, Z2 , V1, V3 = calculate_properties(T, P)
    S11 = R * np.log(Z1 - B)
    S12 = (d_alpha(T) / (2 * np.sqrt(2) * b)) * np.log((Z1 + (1 + np.sqrt(2)) * B) / (Z1 + (1 - np.sqrt(2)) * B))
    
    S21 = R * np.log(Z2 - B)
    S22 = (d_alpha(T) / (2 * np.sqrt(2) * b)) * np.log((Z2 + (1 + np.sqrt(2)) * B) / (Z2 + (1 - np.sqrt(2)) * B))

    # Combine both parts to get the entropy difference
    S1 = S11 + S12
    S2 = S21 + S22
    return S1 , S2

def calc_S(T , P , Trs):
    S1 ,S3 =  delta_S(T, P)
    dS1 = S1 + integral_Cp_by_T(T, Trs)
    dS3 = S3 + integral_Cp_by_T(T, Trs)
    return dS1 , dS3

def compute_integral(P, R, T, a, b, P_min, k=0.001):
    Z1 , Z3 ,V1, V2 = calculate_properties(T, P)
    kappa = 0.37464 + 1.54226*w - 0.26992*w**2 
    alpha = (1 + kappa * (1 - np.sqrt(T / Tc)))**2
    a = a * alpha
    
    # Define the integral function
    def integral_function(V):
        term1 = R * T * np.log(abs(V - b))
        term2 = -(a / (2 * b * np.sqrt(2))) * np.log(abs((V + b - b * np.sqrt(2)) / (V + b + b * np.sqrt(2))))
        return term1 + term2
    
    integral_value = integral_function(V2) - integral_function(V1)
    
    while P > P_min:
        residual = integral_value - (V2 - V1) * P
        if abs(residual) < k:
            return P
        P -= 1  
    
    
    return P



# Define temperature range for plotting

temperature_list = np.arange(314.48,500,10)   # Example temperatures in Kelvin
pressure_values = np.linspace(1, 10e6, 1000)     # Pressure range in bar



# Initialize plot
import plotly.graph_objects as go

fig = go.Figure()

# Loop over each temperature and calculate V for each P
for T in temperature_list:
    
    # Find the local maximum and minimum pressures for the given temperature
    result = minimize_scalar(lambda Vm: -calculate_pressure(Vm, T), bounds=(0.00005, 0.01), method='bounded')
    result_min = minimize_scalar(lambda Vm: calculate_pressure(Vm, T), bounds=(0.00005, 0.01), method='bounded')

    V_max = result.x
    P_max = -result.fun
    V_min = result_min.x
    P_min = -result_min.fun

    # Calculate the saturation pressure for the temperature T
    P_sat = compute_integral(P_max, R, T, a, b, P_min)

    # Generate the PV curve for the temperature T
    Vm = np.linspace(0.00005, 0.01, 1000)  # m^3/mol, narrow around the critical region

    S_values = []
    for P in pressure_values:
        S1 , S3 = calc_S(T, P, Trs)
        S1/=1000
        S3/=1000
        # Check if H is a scalar
        if S1 == S3:
            S_values.append(S1)
        elif P > P_sat:
            S_values.append(S3)
        else:
            S_values.append(S1)

    # Plot the Entropy curve for the current temperature
    fig.add_trace(go.Scatter(x=S_values, y=pressure_values, mode='lines', name=f'T = {T} K'))

# Add labels and legend
fig.update_layout(
    title='Entropy vs Pressure for Formaldehyde at Different Temperatures',
    xaxis_title='Entropy (kJ/mol)',
    yaxis_title='Pressure (Pa)',
    legend_title='Temperature'
)

fig.show()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
